# Modelagem preditiva

A modelagem preditiva são modelos computacionais que usam a matemática para prever resultados futuros. Os algoritmos preditivos usam dados de treino para aprender sobre o comportamento das variáveis afim de buscar a aproximação da função alvo, a função alvo é a formulação matemática que explica a relação entre os dados de entrada e de saída.

A análise preditiva está sendo utilizada por empresas e indivíduos em todo o mundo para extrair valor de dados históricos, após ser desenvolvido e validado, o modelo preditivo consegue generalizar o conhecimento aprendido dos dados históricos para prever o futuro.

### Importando as bibliotecas necessárias

Para fazer nosso trabalho precisamos das funções e métodos por isso vamos importar as bibliotecas do python.

In [7]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from tpot import TPOTRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import RFE

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score 

In [2]:
import warnings
warnings.filterwarnings('ignore')

### Carregando o conjunto de dados

In [3]:
# Carregando o conjunto de dados e salvando na variável 'imoveis'.
imoveis = pd.read_csv('../imoveis_clean_base.csv')

# Vamos tirar as colunas 'Unnamed: 0', 'Link', 'Estado', 'Cidade' e 'Descrição'
imoveis = imoveis.drop(['Unnamed: 0', 'Link', 'Estado', 'Descrição'], axis=1)

In [4]:
pd.set_option('display.max_columns', 100)

# Verificando o dataset
imoveis.head()

,Cidade,Valor_aluguel,Valor_condomínio,Valor_iptu,Área_total,Qt_quartos,Qt_vagas,Qt_banheiros,Academia,Acesso_para_deficientes,Ar_condicionado,Área_de_serviço,Armário_embutido,Armário_na_cozinha,Bicicletário,Churrasqueira,Circuito_de_segurança,Conexão_à_internet,Elevador,Espaço_gourmet,Garagem,Interfone,Lavanderia,Mobiliado,Piscina,Playground,Quadra_de_tênis,Quadra_poliesportiva,Salão_de_festas,Sauna,Segurança_24h,Sistema_de_alarme,Spa,Varanda
0,belo horizonte,3250.0,700.0,0.0,75.0,3.0,2.0,2.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,belo horizonte,3100.0,856.0,290.0,60.0,2.0,2.0,2.0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,belo horizonte,1500.0,280.0,160.0,110.0,3.0,2.0,3.0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1
3,belo horizonte,1300.0,399.0,100.0,60.0,2.0,1.0,2.0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1,1,1,0,0,0,0
4,belo horizonte,1100.0,360.0,110.0,95.0,3.0,2.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0


In [5]:
# Verificando a dimensão do nosso dataset
imoveis.shape

(15958, 34)

In [6]:
df = imoveis

Nesse modelo vamos usar também a variável 'Cidade' porém como essa variável é do tipo texto temos que trata-lá para que o algoritmo possa receber ela da forma certa, em outras palavras como valor numérico. O pacote LabelEncoder transforma rótulos de texto em números.

In [8]:
labelencoder = LabelEncoder()
df['Cidade'] = labelencoder.fit_transform(df['Cidade'])

In [9]:
# Separando a variável target
X = df.drop(['Valor_aluguel'], axis=1)
y = df['Valor_aluguel'].values

In [10]:
X

,Cidade,Valor_condomínio,Valor_iptu,Área_total,Qt_quartos,Qt_vagas,Qt_banheiros,Academia,Acesso_para_deficientes,Ar_condicionado,Área_de_serviço,Armário_embutido,Armário_na_cozinha,Bicicletário,Churrasqueira,Circuito_de_segurança,Conexão_à_internet,Elevador,Espaço_gourmet,Garagem,Interfone,Lavanderia,Mobiliado,Piscina,Playground,Quadra_de_tênis,Quadra_poliesportiva,Salão_de_festas,Sauna,Segurança_24h,Sistema_de_alarme,Spa,Varanda
0,2,700.0,0.0,75.0,3.0,2.0,2.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,856.0,290.0,60.0,2.0,2.0,2.0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,2,280.0,160.0,110.0,3.0,2.0,3.0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1
3,2,399.0,100.0,60.0,2.0,1.0,2.0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1,1,1,0,0,0,0
4,2,360.0,110.0,95.0,3.0,2.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15953,23,300.0,40.0,44.0,1.0,0.0,1.0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15954,23,600.0,0.0,90.0,3.0,1.0,2.0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
15955,23,1200.0,0.0,52.0,2.0,1.0,2.0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
15956,23,350.0,0.0,35.0,1.0,1.0,1.0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
y

array([3250., 3100., 1500., ..., 1600., 1000., 3500.])

In [12]:
# Separando dados de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

#### Classificação de recursos com eliminação recursiva(RFE)

O objetivo da eliminação recursiva de recursos (RFE) é selecionar as melhores variáveis candidatas para o modelo, dado um estimador externo que atribui pesos(importâncias) nas viriáveis, por exemplo, os coeficientes de um modelo linear.

In [13]:
# Treinando um modelo com o algoritmo GradientBoosting e em seguida usando o método RFE para achar as melhores variáveis

model = GradientBoostingRegressor()
model.fit(X_train, y_train)

# Nesse caso abaixo estou considerando as 12 melhores variáveis
rfe = RFE(model, n_features_to_select=12)             
rfe = rfe.fit(X_train, y_train)

In [14]:
features = list(zip(X_train.columns,rfe.support_,rfe.ranking_))

In [15]:
features_true = []
for i in features:
    if 1 in i:
        features_true.append(i)

In [16]:
features_true

[('Cidade', True, 1),
 ('Valor_condomínio', True, 1),
 ('Valor_iptu', True, 1),
 ('Área_total', True, 1),
 ('Qt_vagas', True, 1),
 ('Qt_banheiros', True, 1),
 ('Academia', True, 1),
 ('Ar_condicionado', True, 1),
 ('Elevador', True, 1),
 ('Mobiliado', True, 1),
 ('Piscina', True, 1),
 ('Sauna', True, 1)]

Depois de extrair as variáveis estou imprimindo elas.

In [28]:
cols = []

for i in range(0,len(features_true)):
    cols.append(features_true[i][0])
    
cols

['Cidade',
 'Valor_condomínio',
 'Valor_iptu',
 'Área_total',
 'Qt_vagas',
 'Qt_banheiros',
 'Academia',
 'Ar_condicionado',
 'Elevador',
 'Mobiliado',
 'Piscina',
 'Sauna']

Filtrando nosso dataset somente com a variáveis escolhidas pelo RFE.

In [29]:
df[cols].head()

,Cidade,Valor_condomínio,Valor_iptu,Área_total,Qt_vagas,Qt_banheiros,Academia,Ar_condicionado,Elevador,Mobiliado,Piscina,Sauna
0,2,700.0,0.0,75.0,2.0,2.0,0,0,1,0,0,0
1,2,856.0,290.0,60.0,2.0,2.0,1,0,1,0,1,0
2,2,280.0,160.0,110.0,2.0,3.0,0,0,0,0,0,0
3,2,399.0,100.0,60.0,1.0,2.0,1,0,1,0,1,1
4,2,360.0,110.0,95.0,2.0,1.0,0,0,0,0,0,0


In [30]:
X = df[cols]
y = df['Valor_aluguel'].values

### Criando dados de treino e teste

In [31]:
# Separando dados de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

### Modelo com o Tpot AutoML

O objetivo do TPOT é automatizar a construção de pipelines de machine learning combinando uma representação de árvore de expressão flexível de pipelines com algoritmos de pesquisa. O TPOT usa a biblioteca scikit-learn baseada em Python.

In [32]:
model = TPOTRegressor(generations=2, 
                      population_size=100, 
                      scoring='r2', 
                      cv=2, verbosity=2, 
                      random_state=0)

In [33]:
%%time
model.fit(X_train, y_train)

[08:10:46] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:10:48] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:10:49] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:10:52] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:10:52] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:10:53] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:10:54] WARNI

[08:11:03] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:11:05] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:11:36] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:11:36] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:12:16] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:12:16] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:12:39] WARNI

[08:18:28] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:18:30] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:18:45] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:18:46] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:19:21] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:19:23] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:19:32] WARNI

TPOTRegressor(config_dict=None, crossover_rate=0.1, cv=2,
              disable_update_check=False, early_stop=None, generations=2,
              max_eval_time_mins=5, max_time_mins=None, memory=None,
              mutation_rate=0.9, n_jobs=1, offspring_size=None,
              periodic_checkpoint_folder=None, population_size=100,
              random_state=0, scoring='r2', subsample=1.0,
              template='RandomTree', use_dask=False, verbosity=2,
              warm_start=False)

### Exportando a pipeline de execução

In [23]:
model.export('pipeline.py')
with open('pipeline.py') as pipe:
    print("".join(pipe.readlines()))

import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

# NOTE: Make sure that the class is labeled 'target' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
features = tpot_data.drop('target', axis=1).values
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['target'].values, random_state=0)

# Average CV score on the training set was:0.6432237069192137
exported_pipeline = GradientBoostingRegressor(alpha=0.99, learning_rate=0.1, loss="ls", max_depth=5, max_features=0.5, min_samples_leaf=11, min_samples_split=9, n_estimators=100, subsample=0.8)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)



### Executando a pipeline

In [24]:
model_tpot = GradientBoostingRegressor(alpha=0.99, learning_rate=0.1, loss="ls", max_depth=5, max_features=0.5,\
                                       min_samples_leaf=11, min_samples_split=9, n_estimators=100, subsample=0.8)
model_tpot.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.99, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=5,
                          max_features=0.5, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=11, min_samples_split=9,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=0.8, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [25]:
result = model_tpot.predict(X_test)
result

array([1868.821151  , 2356.1596793 , 1557.39170916, ...,  777.40930522,
       2494.8198722 , 2636.48952945])

In [26]:
R2 = r2_score(y_test,result)
MAE = mean_absolute_error(y_test,result)
print(R2)
print(MAE)

0.6442432268539305
370.2656133985721


-------------------------------

## Métricas de avaliação

### R-Quadrado (R²)

O R-Quadrado, ou Coeficiente de Determinação, é uma métrica que visa expressar a quantidade da variança dos dados que é explicada pelo modelo. Em outras palavras, essa medida calcula qual a porcentagem da variança que pôde ser prevista pelo modelo de regressão e, portanto, nos diz o quão “próximo” as medidas reais estão do nosso modelo.

O valor do seu R-Quadrado varia de 0 a 1 e geralmente é representado em porcentagem. Por exemplo, um R² = 75% nos diz que 75% da variância de nossos dados podem ser explicados pelo modelo construído, enquanto os outros 25%, teoricamente, se tratariam de uma variância residual.

### Erro Absoluto Médio (MAE)

O Erro Absoluto Médio consiste na média das distâncias entre valores preditos e reais. Em outras palavras, tira-se média da diferença absoluta dos valores preditos e real.

In [ ]:
# No exemplo abaixo vamos ver o MSE do algoritmo XGB, pois no loop for foi o último a ser executado, na matemática fica assim:
(abs(y_test - result).sum() / len(result))

### Erro Quadrático Médio (MSE)

O Erro Quadrático Médio consiste na média do erro das previsões ao quadrado. Em outras palavras, é a média da soma dos resíduos(diferença entra os valores predito e real) ao quadrado.

In [ ]:
# No exemplo abaixo vamos ver o MSE do algoritmo XGB, pois no loop for foi o último a ser executado, na matemática fica assim:
((y_test - result)**2).sum() / len(result)

------------------------------------------------------

### Melhor resultado do modelo:

In [27]:
R2 = r2_score(y_test,result)
MAE = mean_absolute_error(y_test,result)
print(R2)
print(MAE)

0.6442432268539305
370.2656133985721
